In [1]:
#!/usr/bin/env python
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np, os, sys
from tqdm.notebook import tqdm    
from manipulations import get_classes, get_classes_from_header, get_Fs_from_header, load_challenge_data
from sklearn.model_selection import KFold, StratifiedKFold

In [2]:
N_CLASS = 27

In [3]:
MAX_RR = 750 # 60 beats/min => 60 beats/60 s ==> beat/1s ==> 500 samples / beat

In [4]:
from manipulations import get_abbr, get_name
from global_vars import labels, Dx_map, Dx_map_unscored
first_idx = {scored_code: None for scored_code in list(Dx_map['SNOMED CT Code'])}
first_idx_unscored = {unscored_code: None for unscored_code in list(Dx_map_unscored['SNOMED CT Code'])}

In [5]:
from global_vars import labels, normal_class, equivalent_mapping
normal_idx = np.argwhere(labels==int(normal_class))
def get_scored_class(code, labels):
    return [1 if label in code else 0 for label in labels]

In [6]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
#pip install iterative-stratification

if __name__ == '__main__':
    
    Datas = []
    Header_datas = []
    Classes = []
    Codes = []
    
    dataset_idx = {}
    dataset_data_labels = {}
    dataset_train_idx = {}
    dataset_test_idx = {}
    
    global_idx = 0
    datasets = [1,2,3,4,5,6]
    for dataset in datasets:
        print('Dataset ', dataset)
        # Parse arguments.
        if len(sys.argv) != 3:
            raise Exception('Include the input and output directories as arguments, e.g., python driver.py input output.')

        input_directory = '../NewData/{}/'.format(dataset)
        output_directory = '../Output/'

        # Find files.
        input_files = []
        for f in os.listdir(input_directory):
            if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
                input_files.append(f)

        if not os.path.isdir(output_directory):
            os.mkdir(output_directory)

        classes=get_classes(input_directory,input_files)

        num_files = len(input_files)
        datas = []
        header_datas = []
        dataset_data_labels[dataset] = []
        dataset_idx[dataset] = []
        for i, f in tqdm(enumerate(input_files)):
            #print('    {}/{}...'.format(i+1, num_files), f)
            tmp_input_file = os.path.join(input_directory,f)
            data,header_data = load_challenge_data(tmp_input_file)
            
            codes = get_classes_from_header(header_data)
            data_labels = get_scored_class(codes, labels)
            
            datas.append(data[:,1000:7000])
            header_datas.append(header_data)
            dataset_data_labels[dataset].append(data_labels)
            dataset_idx[dataset].append(global_idx)
            global_idx += 1

        Datas += datas
        Header_datas += header_datas
        Classes += classes
        
        kf = MultilabelStratifiedKFold(5)
        train_idx, test_idx = next(kf.split(datas, np.array(dataset_data_labels[dataset])))


        dataset_train_idx[dataset] = train_idx
        dataset_test_idx[dataset] = test_idx
        
        
        print('Done.')

Dataset  1



Done.
Dataset  2



Done.
Dataset  3



Done.
Dataset  4



Done.
Dataset  5



Done.
Dataset  6



Done.


In [9]:
import pickle
Q_locs = None
with open('../saved/newData_Q_locs_1000_7000_peakdist100.pkl', 'rb') as Q_locs_file:
    Q_locs = pickle.load(Q_locs_file)

Codes = None
with open('../saved/newData_Codes_1000_7000_peakdist100.pkl', 'rb') as Codes_file:
    Codes = pickle.load(Codes_file)

In [106]:
with open('../saved/all_train_idx_stratified.pkl', 'rb') as all_train_idx_file:
    all_train_idx = pickle.load(all_train_idx_file)
with open('../saved/all_test_idx_stratified.pkl', 'rb') as all_test_idx_file:
    all_test_idx = pickle.load(all_test_idx_file)

In [109]:
len(Idxes_test)

52864

In [110]:
X_test = []
Data_labels_test = []
Idxes_test = []
for i in tqdm(all_test_idx):
    #print(i)
        
    Q_loc = Q_locs[i]
    RR_avg = np.median([Q_loc[k+1] - Q_loc[k] for k in range(len(Q_loc)-1)])
    RR_th = (0.3 * RR_avg, 3 * RR_avg)
    
    ks = [k for k in range(len(Q_loc)-1) if Q_loc[k+1] - Q_loc[k] > RR_th[0] 
              and Q_loc[k+1] - Q_loc[k] < RR_th[1]]
    
    
    for k in ks:
        Data_labels_test.append(get_scored_class(Codes[i], labels))
        X_test.append(Datas[i][:,1000+Q_loc[k]:1000+Q_loc[k+1]])
        Idxes_test.append(i)
        

In [111]:
Idxes_test

[4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 55,
 55,
 55,
 55,
 55,
 55,
 55,
 55,
 55,
 55,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 83,
 83,
 83,
 83,
 83,
 83,
 83,
 83,
 83,
 83,
 84,
 84,
 84,
 84,
 84,
 84,
 84,
 84,
 84,
 89,
 89,
 89,
 89,
 89,
 89,

# On y va

In [21]:
Signals_test = np.load('../saved/Signals_test_stratified.npy')

In [22]:
import pickle
with open('../saved/Data_labels_test_stratified.pkl', 'rb') as Data_labels_test_file:
    Data_labels_test = pickle.load(Data_labels_test_file)

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from resnet1d import ECGResNet50

def load_trained_model(model_saved_path):
    model = ECGResNet50(MAX_RR, N_CLASS)
    
    # load saved model
    model.load_state_dict(torch.load(model_saved_path, map_location=torch.device('cpu')))
    model.eval()
    return model

In [24]:
model_saved_path = '../saved/resnet50/resnet50_resnet50_minibatch_adam_batch512_0.01_stratified_model.dict'
model = load_trained_model(model_saved_path)

In [25]:
import torch
from torch import nn

In [26]:
Signal_12leads_test= np.transpose(Signals_test, (1,0,2))

In [27]:
from torch.utils.data import Dataset
from torchvision import transforms

import os
class SignalDataset(Dataset):

    def __init__(self, signals, labels):
        self.signals = signals
        self.labels = labels

    def __len__(self):
        return len(self.signals)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample =(torch.cat([torch.Tensor(np.array([self.signals[channel,idx]]).transpose()) for channel in range(12)], axis=1), 
                  torch.Tensor(self.labels[idx]))

        return sample
    
signal_datasets_test = SignalDataset(Signal_12leads_test, np.array(Data_labels_test)[:,:N_CLASS])


In [28]:
assert np.array(Data_labels_test).shape[1] == N_CLASS

In [29]:
from evaluation.evaluate_12ECG_score import compute_modified_confusion_matrix, compute_challenge_metric

In [30]:
def compute_score(labels, outputs, weights, normal_index=normal_idx):
    num_recordings, num_classes = np.shape(labels)
    # Compute the observed score.
    A = compute_modified_confusion_matrix(labels, outputs)
    observed_score = np.nansum(weights * A)

    # Compute the score for the model that always chooses the correct label(s).
    correct_outputs = labels
    A = compute_modified_confusion_matrix(labels, correct_outputs)
    correct_score = np.nansum(weights * A)

    # Compute the score for the model that always chooses the normal class.
    inactive_outputs = np.zeros((num_recordings, num_classes), dtype=np.bool)
    inactive_outputs[:, normal_index] = 1
    A = compute_modified_confusion_matrix(labels, inactive_outputs)
    inactive_score = np.nansum(weights * A)

    if correct_score != inactive_score:
        normalized_score = float(observed_score - inactive_score) / float(correct_score - inactive_score)
    else:
        normalized_score = float('nan')

    return normalized_score

In [31]:
from torchvision import datasets, models, transforms
from myeval import agg_y_preds_bags, binary_acc, geometry_loss
import torch.optim as optim
from torch.optim import lr_scheduler
from snippets.pytorchtools import EarlyStopping
from sklearn.model_selection import GroupKFold
import time

st = time.time()
patience = 50
batch_size= 512#65000

saved_dir = '../saved/resnet50/'


testDataset = torch.utils.data.Subset(signal_datasets_test, range(0,len(Signals_test), 1))

testLoader = torch.utils.data.DataLoader(testDataset, batch_size=65000, shuffle = False, pin_memory=True)



In [32]:
np.array(Signals_test).shape

(94421, 12, 750)

In [33]:
names = [get_name(label, Dx_map, Dx_map_unscored) for label in labels]

assert len(labels) == 27

In [34]:
y_tests_tensor = None

output_tests = []
y_tests = []
with torch.no_grad():

    for X_test, y_test in tqdm(testLoader):  
        output_test = model(X_test)
        output_tests.append(output_test)
        y_tests.append(y_test)

    y_tests_tensor = torch.cat(y_tests, axis=0) # ground truth

    output_tests = torch.cat(output_tests, axis=0)
    y_test_preds = torch.sigmoid(output_tests)

#     output_trains = torch.cat(output_trains, axis=0)
#     y_train_preds_max, y_train_preds_mean, _ = agg_y_preds_bags(y_train_preds, bag_size=n_segments)
#     y_test_preds_max, y_test_preds_mean, _ = agg_y_preds_bags(y_test_preds, bag_size=n_segments)
#     _, _, y_trains = agg_y_preds_bags(y_trains, bag_size=n_segments)
#     _, _, y_tests = agg_y_preds_bags(y_tests, bag_size=n_segments)


In [35]:
from global_vars import weights
acc2, fmeasure2, fbeta2, gbeta2 = binary_acc(y_test_preds, y_tests_tensor)
geometry2 = geometry_loss(fbeta2, gbeta2)

score2 = compute_score(np.round(y_test_preds.data.numpy()), np.round(y_tests_tensor.data.numpy()), weights)
output_str = 'End {:.2f} min |\n Valid: Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, geo: {:.3f}, score: {:.3f}\n '.format(
    (time.time()-st)/60, acc2, fmeasure2, fbeta2, gbeta2, geometry2, score2)

print(output_str)

#     output_string = 'AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure|Geomotry\n{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}'.format(auroc2,auprc2,acc2,fmeasure2,fbeta2,gbeta2,geometry2)
#     print(output_string)     
#     with open(saved_dir+'score'+ str(i)+ '_epoch' + str(epoch) + '.txt', 'w') as f:
#         f.write(output_string)

#     avg_losses_train = np.array(avg_losses_train)
#     avg_losses_test = np.array(avg_losses_test)

#     np.save(saved_dir + 'avg_losses_train' + str(i) + '_epoch' + str(epoch), avg_losses_train)
#     np.save(saved_dir + 'avg_losses_test' + str(i) + '_epoch' + str(epoch), avg_losses_test)


End 1.87 min |
 Valid: Acc: 0.979, F: 0.264, Fbeta: 0.289, gbeta: 0.152, geo: 0.210, score: 0.533
 


In [36]:
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix
from global_vars import labels

cf_matrices = multilabel_confusion_matrix(y_tests_tensor.data.numpy(), np.round(y_test_preds.data.numpy()))

In [37]:
for label, cf_matrix in zip(labels, cf_matrices):
    print(get_name(label, Dx_map, Dx_map_unscored))
    print(cf_matrix)
    print('\n')


1st degree av block
[[86789  1221]
 [ 2103  4308]]


atrial fibrillation
[[77102  2966]
 [ 3969 10384]]


atrial flutter
[[93978    34]
 [  336    73]]


bradycardia
[[92861   548]
 [  665   347]]


complete right bundle branch block
[[93151   301]
 [  725   244]]


incomplete right bundle branch block
[[92592   473]
 [ 1151   205]]


left anterior fascicular block
[[92720   354]
 [  984   363]]


left axis deviation
[[89543  1218]
 [ 2564  1096]]


left bundle branch block
[[91556   374]
 [  752  1739]]


low qrs voltages
[[94209     3]
 [  209     0]]


nonspecific intraventricular conduction disorder
[[93758    16]
 [  646     1]]


pacing rhythm
[[94215    61]
 [  126    19]]


premature atrial contraction
[[86387  1564]
 [ 2991  3479]]


premature ventricular contractions
[[93251   202]
 [  897    71]]


prolonged pr interval
[[93971    42]
 [  406     2]]


prolonged qt interval
[[94216     0]
 [  205     0]]


qwave abnormal
[[93856    52]
 [  506     7]]


right axis deviation


In [38]:
from signal_processing import find_AD

In [39]:
ads = []
for signal, label in tqdm(signal_datasets_test):
    #print(signal.numpy().shape)
    ad = find_AD(signal.numpy().transpose())
    ads.append(ad)

In [40]:
test_pred_labels = np.round(y_test_preds.data.numpy())

In [41]:
test_pred_labels.shape

(94421, 27)

In [42]:
from global_vars import lad_class, rad_class
lad_idx = np.argwhere(labels == int(lad_class)).flatten()
rad_idx = np.argwhere(labels == int(rad_class)).flatten()

In [43]:
labels

array([270492004, 164889003, 164890007, 426627000, 713427006, 713426002,
       445118002,  39732003, 164909002, 251146004, 698252002,  10370003,
       284470004, 427172004, 164947007, 111975006, 164917005,  47665007,
        59118001, 427393009, 426177001, 426783006, 427084000,  63593006,
       164934002,  59931005,  17338001])

In [44]:
for i, ad in enumerate(ads):
    if ad == 'LAD':
        test_pred_labels[i, lad_idx] = 1
        test_pred_labels[i, rad_idx] = 0
    elif ad == 'RAD':
        test_pred_labels[i, rad_idx] = 1
        test_pred_labels[i, lad_idx] = 0
    else:
        test_pred_labels[i, rad_idx] = 0
        test_pred_labels[i, lad_idx] = 0

In [45]:
cf_matrices = multilabel_confusion_matrix(y_tests_tensor.data.numpy(), test_pred_labels)

In [46]:
for label, cf_matrix in zip(labels, cf_matrices):
    print(get_name(label, Dx_map, Dx_map_unscored))
    print(cf_matrix)
    print('\n')


1st degree av block
[[86789  1221]
 [ 2103  4308]]


atrial fibrillation
[[77102  2966]
 [ 3969 10384]]


atrial flutter
[[93978    34]
 [  336    73]]


bradycardia
[[92861   548]
 [  665   347]]


complete right bundle branch block
[[93151   301]
 [  725   244]]


incomplete right bundle branch block
[[92592   473]
 [ 1151   205]]


left anterior fascicular block
[[92720   354]
 [  984   363]]


left axis deviation
[[69766 20995]
 [ 1217  2443]]


left bundle branch block
[[91556   374]
 [  752  1739]]


low qrs voltages
[[94209     3]
 [  209     0]]


nonspecific intraventricular conduction disorder
[[93758    16]
 [  646     1]]


pacing rhythm
[[94215    61]
 [  126    19]]


premature atrial contraction
[[86387  1564]
 [ 2991  3479]]


premature ventricular contractions
[[93251   202]
 [  897    71]]


prolonged pr interval
[[93971    42]
 [  406     2]]


prolonged qt interval
[[94216     0]
 [  205     0]]


qwave abnormal
[[93856    52]
 [  506     7]]


right axis deviation


In [47]:
score2 = compute_score(np.round(y_test_preds.data.numpy()), test_pred_labels, weights)


In [48]:
score2

0.785374733086021

In [92]:
def aggr_bag_res(test_pred_labels, Idxes_test):
    assert len(test_pred_labels) == len(Idxes_test)
    uniq = np.unique(Idxes_test, return_counts=True)[1]
    cumsum = np.cumsum(uniq)[:-1]
    res = np.split(test_pred_labels, cumsum)
    print("Done")
    return res, cumsum, uniq
#     aggr_labels = []
#     cur_label = np.zeros((N_CLASS,))
#     for idx in Idxes_test:
#         cur_label += test_pred_labels[idx,:]
#         aggr_labels = (cur_label > 0).astype(int)
    

agg_res, cumsum, uniq= aggr_bag_res(Idxes_test, Idxes_test)

Done


In [113]:
Idxes_test

[4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 55,
 55,
 55,
 55,
 55,
 55,
 55,
 55,
 55,
 55,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 83,
 83,
 83,
 83,
 83,
 83,
 83,
 83,
 83,
 83,
 84,
 84,
 84,
 84,
 84,
 84,
 84,
 84,
 84,
 89,
 89,
 89,
 89,
 89,
 89,

In [94]:
Idxes_test[18]

30

In [112]:
np.unique(Idxes_test, return_counts=True)

(array([    4,    11,    13, ..., 43078, 43079, 43086]),
 array([24, 13, 12, ..., 11,  9,  6]))

In [102]:
1 in Idxes_test

True

In [96]:
np.unique(Idxes_test, return_counts=True)
cumsum = np.cumsum(uniq)[:-1]
res = np.split(Idxes_test, cumsum)

In [97]:
len(X_test)

94267

In [98]:
len(res[0])

18